In [1]:
!pip install tensorflow

In [2]:
# Import the necessary packages
from tensorflow.keras.utils import normalize
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

In [3]:
def build_model(width, height, depth, classes):
  # Keras' Sequential API
  model = Sequential()

  # First set of CONV => RELU => POOL layers
  model.add(Conv2D(32, (5, 5), padding="same", input_shape=(height, width, depth)))
  model.add(Activation("relu"))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  # Second set of CONV => RELU => POOL layers
  model.add(Conv2D(32, (3, 3), padding="same"))
  model.add(Activation("relu"))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  # First set of FC => RELU layers
  model.add(Flatten())
  model.add(Dense(64))
  model.add(Activation("relu"))
  model.add(Dropout(0.5))
  
  # Second set of FC => RELU layers
  model.add(Dense(64))
  model.add(Activation("relu"))
  model.add(Dropout(0.5))
  
  # Softmax classifier
  model.add(Dense(classes))
  model.add(Activation("softmax"))
  
  # Return the constructed model
  return model

In [5]:
# Initialize the learning rate, number of epochs and batch size
LR = 1e-3
EPOCHS = 10
BATCH_SIZE = 128


# Load the MNIST dataset
print("[PROGRESS] Loading MNIST dataset...")
(x_train, y_train), (x_test, y_test) = mnist.load_data()


print("\n[PROGRESS] Data transformation")
# Add a channel dimension to the digits since they dont have one
# Channel dimension = 1 (i.e., grayscale)
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

# Check range of values
print('x_train - Min: %.3f, Max: %.3f' % (x_train.min(), x_train.max()))
print('x_test - Min: %.3f, Max: %.3f' % (x_test.min(), x_test.max()))

# Scale data to range of [0, 1]
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0 
x_test /= 255.0

# Verify that the data has been scaled
print('\nScaled x_train - Min: %.3f, Max: %.3f' % (x_train.min(), x_train.max()))
print('Scaled x_test -Min: %.3f, Max: %.3f' % (x_test.min(), x_test.max()))

# Convert the labels from integers to vectors
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)


# Initialize the optimizer and model
print("\n[PROGRESS] Compiling model...")
model = build_model(width=28, height=28, depth=1, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=LR), metrics=["accuracy"])


# Train the network
print("\n[PROGRESS] Training network...")
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=EPOCHS, batch_size=BATCH_SIZE)


# Evaluate the network
print("\n[PROGRESS] Evaluating network...")
y_pred = model.predict(x_test)
print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1)))


# Serialize the model to disk
print("\n[PROGRESS] Serializing digit model...")
model.save("digit_recognition.h5", save_format="h5")

[PROGRESS] Loading MNIST dataset...

[PROGRESS] Data transformation
x_train - Min: 0.000, Max: 255.000
x_test - Min: 0.000, Max: 255.000

Scaled x_train - Min: 0.000, Max: 1.000
Scaled x_test -Min: 0.000, Max: 1.000

[PROGRESS] Compiling model...

[PROGRESS] Training network...
Epoch 1/10
469/469 [==============================] - 60s 127ms/step - loss: 0.7726 - accuracy: 0.7365 - val_loss: 0.0941 - val_accuracy: 0.9740
Epoch 2/10
469/469 [==============================] - 59s 126ms/step - loss: 0.2776 - accuracy: 0.9167 - val_loss: 0.0593 - val_accuracy: 0.9824
Epoch 3/10
469/469 [==============================] - 59s 125ms/step - loss: 0.2045 - accuracy: 0.9392 - val_loss: 0.0482 - val_accuracy: 0.9856
Epoch 4/10
469/469 [==============================] - 59s 125ms/step - loss: 0.1750 - accuracy: 0.9487 - val_loss: 0.0459 - val_accuracy: 0.9854
Epoch 5/10
469/469 [==============================] - 59s 125ms/step - loss: 0.1580 - accuracy: 0.9526 - val_loss: 0.0427 - val_accuracy: 0.9